# **Importing Libraries**

In [5]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
from matplotlib import pyplot
import os
import pickle
import random
from random import shuffle
import tensorflow as tf
import tensorflow_datasets as tfds
import torch
import cv2
from tqdm import tqdm
from PIL import Image
import PIL

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Loading Datasets to Colab**

In [3]:
from zipfile import ZipFile

In [4]:
# with ZipFile('/content/drive/MyDrive/Graduation_Project/Datasets/UTK_DS_notPreprocessed.zip', 'r') as zipObj:
#   zipObj.extractall('')

with ZipFile('/content/drive/MyDrive/Graduation_Project/Datasets/Audience_DS_Kaggle.zip', 'r') as zipObj:
  zipObj.extractall('')
with ZipFile('/content/drive/MyDrive/Graduation_Project/Datasets/Pictures_age_4_13.zip', 'r') as zipObj:
  zipObj.extractall('')

In [ ]:
# whole_paths = np.load('/content/drive/MyDrive/ImagesSavedfromColab/Audience_DS_kaggle.npy', allow_pickle='TRUE')
# # # whole_paths = np.load('/content/drive/MyDrive/ImagesSavedfromColab/UTK_DS.npy', allow_pickle='TRUE')
# print(whole_paths.shape)

(18622, 2)


# **Save a numpy array image to an image in the drive**

In [ ]:
def list_images_paths(basePath, validExts=(".jpg", ".jpeg", ".png", ".bmp",".JPG"), contains=None):

    # loop over the directory structure
    for (rootDir, dirNames, filenames) in os.walk(basePath):
        # loop over the filenames in the current directory
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
              print("contains is not None and filename.find(contains) == -1")
              continue

            # determine the file extension of the current file
            try:

              age = str(filename.split('_')[0])
              # print(age)
            except Exception:
              print('ERROR IN CONVERTING THE AGE TO INT')
              continue



            ext = filename[filename.rfind("."):].lower()

            # check to see if the file is an image and should be processed
            if ext.endswith(validExts):
              # construct the path to the image and yield it
              imagePath = os.path.join(rootDir, filename)#.replace(" ", "\\ ")
              print(imagePath)
              yield [imagePath, age]



> Private_ds



In [ ]:
PrivatePaths = list(list_images_paths('/content/Private_ds'))

In [ ]:
print(len(PrivatePaths))
Private_paths = np.array(PrivatePaths)
print(Private_paths.shape)

1079
(1079, 2)


In [ ]:
np.save('Private_DS.npy', Private_paths)
%cp /content/Private_DS.npy /content/drive/MyDrive/ImagesSavedfromColab/Private_DS.npy



> Adience_ds



In [ ]:
AudiencePaths = list(list_images_paths('/content/content/Aduience_ds'))
print(len(AudiencePaths))
Audience_paths = np.array(AudiencePaths)
print(Audience_paths.shape)

In [ ]:
np.save('Audience_DS.npy', Audience_paths)
%cp /content/Audience_DS.npy /content/drive/MyDrive/ImagesSavedfromColab/Audience_DS_kaggle.npy



> UTK_ds



In [ ]:
UTKPaths = list(list_images_paths('/content/content/Preprocessed_UTK_DS'))

In [ ]:
print(len(UTKPaths))
UTK_paths = np.array(UTKPaths)
print(UTK_paths.shape)

In [ ]:
np.save('UTK_DS.npy', UTK_paths)
%cp /content/UTK_DS.npy /content/drive/MyDrive/ImagesSavedfromColab/UTK_DS.npy

In [ ]:
# Visualization Cell
fig = plt.figure(figsize=(9, 13))
columns = 4
rows = 5
# for i in range(1, columns*rows +1):
#     img = np.random.randint(10, size=(h,w))
#     fig.add_subplot(rows, columns, i)
#     plt.imshow(img)
# plt.show()
xs = np.linspace(0, 2*np.pi, 60)  # from 0 to 2pi
ys = np.abs(np.sin(xs))
ax = []
idx=1
for i in Private_paths[:20]:
    path = i

    path[0] = path[0].replace('\\','/')
    print(path[0])
    image=PIL.Image.open(path[0])

    ax.append(fig.add_subplot(rows, columns, idx))
    age = path[1]
    ax[-1].set_title("age:"+str(age))
    plt.imshow(image)
    # image = cv2.imread(path[0])
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)



    #print("Images Shape = ", image.size)

    #print("Ages = ", age)
    #print("idx= ",idx)
    idx+=1
    # pyplot.imshow(image)
    # pyplot.show()
ax[2].plot(xs, 3*ys)
ax[19].plot(ys**2, xs)
plt.show()

# **Preprocessing**

In [ ]:
%cp  /content/drive/MyDrive/Graduation_Project/Weights/yolov5s-face.onnx  /content/yolov5s_face.onnx
%cp /content/drive/MyDrive/Graduation_Project/Notebooks/face_detecion&aligned.py         /content/face_detection.py

In [ ]:
%cd /content

/content


In [ ]:
from face_detection import yolov5

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def image_enhancement(image):

    # Denosing
    image = cv2.fastNlMeansDenoisingColored(np.uint8(image),None,5,10,7,21)

    return image

In [ ]:
def image_preprocessing(image):

    # Enhancement
    #image = image_enhancement(image)

    # Alignment and Cropping
    image = yolov5(image, model_path='yolov5s_face.onnx')

    if image is None:
        print('Empty Image from Alignment and Cropping')
        return None

    # # Image size for our model.
    # MODEL_INPUT_IMAGE_SIZE = [ 96 , 96 ]

    # # Resizing
    # image = tf.image.resize(image, MODEL_INPUT_IMAGE_SIZE)

    # # Normalization
    # image = image/255.0

    return image # Return image after resizin

In [ ]:
%mkdir /content/preprocessed_Audience_DS #create folder to save preprocessed image

In [ ]:
i = 0
for path in tqdm(whole_paths[:10000]):

  path[0] = path[0].replace('\\','/')
  try:
    image = cv2.imread(path[0])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  except Exception:
    print('ERROR IN reading image')
    continue
  image = image_preprocessing(image) # PREPROCESSING
  if image is not None:
         # Save the image after preprocessing to a folder
         image = np.array(image) # Not sure to add this line or not
         image = PIL.Image.fromarray(image)
        #  pyplot.imshow(image)
        #  pyplot.show()
        #  break
         age = path[1]

         # Create this folder before run the cell--> Preprocessed_Merged_DS
         image_filename = '/content/preprocessed_Audience_DS/' + str(age) + '_age_' + str(i) +'_ID.jpg' # IMAGE PATH
         image.save(image_filename)
         #print(image_filename)
  # if i == 10000:
  #    break

  i+=1


100%|██████████| 1/1 [00:01<00:00,  1.12s/it]


In [ ]:
# # CONVERT THE FOLDER INTO ZIP FILE
!zip -r Preprocessed_Audience_DS.zip /content/preprocessed_Audience_DS

In [ ]:
# # MOVE THE ZIP FOLDER TO THE DRIVE
%cp /content/Preprocessed_Audience_DS.zip /content/drive/MyDrive/ImagesSavedfromColab/Preprocessed_Audience_DS_Kaggle.zip